In [1]:
from pathlib import Path
import json

import pandas as pd
import numpy as np
from tqdm import tqdm

#from statsmodels.stats.proportion import proportions_ztest, proportions_ztost, test_proportions_2indep
#from statsmodels.stats.nonparametric import rank_compare_2indep
#from statsmodels.stats.weightstats import ttest_ind, ttost_ind
from scipy.stats import pearsonr, spearmanr, mannwhitneyu, ttest_rel

In [3]:
import numpy as np
from sklearn import metrics

def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix) 

First, using ALTO results, let's get the diff in probability we need to be sensitive to

In [4]:
all_pps = list(np.arange(0, 1, 0.001))
all_purities = []
num_docs = 200
num_topics = 20
l_topics = list(np.arange(num_topics) + 1)

for pp in tqdm(all_pps):
    probs_list = [pp] + [(1 - pp)/(num_topics - 1)]*(num_topics - 1)
    ground_truth = np.random.randint(num_topics, size=num_docs) + 1
    
    annotator_vec = []
    for doc_i in range(num_docs):
        gt = ground_truth[doc_i]
        annotator_vec.append(np.random.choice([gt] + l_topics[:(gt-1)] + l_topics[gt:], 
                                        p=probs_list))
    all_purities.append(purity_score(ground_truth, annotator_vec))

100%|██████████| 1000/1000 [00:05<00:00, 198.35it/s]


In [6]:
alto_b_purity = 0.35371
b_diffs = []
for p in all_purities:
    b_diffs.append(abs(p - alto_b_purity))
b_ind = np.argmin(b_diffs)
selected_b_purity = all_purities[b_ind]
print('Prob of binning a doc correctly for better condition that gives closest result to ALTO = ' + str(selected_b_purity))

Prob of binning a doc correctly for better condition that gives closest result to ALTO = 0.355


In [7]:
alto_w_purity = 0.32384
w_diffs = []
for p in all_purities:
    w_diffs.append(abs(p - alto_w_purity))
w_ind = np.argmin(w_diffs)
selected_w_purity = all_purities[w_ind]
print('Prob of binning a doc correctly for worse condition that gives closest result to ALTO = ' + str(selected_w_purity))

Prob of binning a doc correctly for worse condition that gives closest result to ALTO = 0.325


In [8]:
prob_diff = selected_b_purity - selected_w_purity
print('Prob diff we want to be sensitive to = ' + str(prob_diff))

Prob diff we want to be sensitive to = 0.02999999999999997


In [9]:
def rand_bin_array(topics, questions=30):
    selections = np.zeros(topics)
    selections[:questions]  = 1
    np.random.shuffle(selections)
    return selections

def permutation_test(a, b, alternative="two-sided", value=0, iters=1000):
    og_diff = np.mean(a) - np.mean(b) - value
    combin = np.concatenate([a, b])
    n_a, n_b = len(a), len(b)
    diffs = [
        np.mean(samp[:n_a]) - np.mean(samp[n_a:])
        for _ in range(iters)
        for samp in [np.random.choice(combin, n_a + n_b, replace=False)]
    ]
    if alternative == "two-sided":
        return og_diff, np.mean(np.abs(og_diff) < np.abs(diffs)), diffs
    elif alternative == "larger":
        return og_diff, np.mean(og_diff < diffs), diffs
    elif alternative == "smaller":
        return og_diff, np.mean(og_diff > diffs), diffs
    else:
        raise ValueError("alternative must be one of (two-sided, larger, smaller)")

In [10]:
def run_sims(num_annotators, prob_diff):   
    
    pvals = []
    num_docs = 200 #len(set(nist_ras_data['doc_id'])) #assuming a sample of all docs is shown?
    num_topics = 20
    l_topics = list(np.arange(num_topics) + 1)
    
    #prob_diff = 0.1 #effect size - probability difference between the two scenarios for picking right label 
    num_iters = 10000

    np.random.seed(454)
    for _ in tqdm(range(num_iters)):
        pp_b = np.random.uniform(0.1, 1)
        probs_list_b = [pp_b] + [(1 - pp_b)/(num_topics - 1)]*(num_topics - 1)
        pp_w = pp_b - prob_diff
        probs_list_w = [pp_w] + [(1 - pp_w)/(num_topics - 1)]*(num_topics - 1)
        
        ground_truth = np.random.randint(num_topics, size=num_docs) + 1
        
        s1_vals, s2_vals = [], []
        for _ in range(num_annotators):
            a_1_vec, a_2_vec = [], []
            for doc_i in range(num_docs):
                gt = ground_truth[doc_i]
                a_1_vec.append(np.random.choice([gt] + l_topics[:(gt-1)] + l_topics[gt:], 
                                                p=probs_list_b))
                a_2_vec.append(np.random.choice([gt] + l_topics[:(gt-1)] + l_topics[gt:], 
                                                p=probs_list_w))
            s1_vals.append(purity_score(ground_truth, a_1_vec))
            s2_vals.append(purity_score(ground_truth, a_2_vec))
        
        stat, pval, _ = permutation_test(s1_vals, s2_vals, alternative = 'larger')
        #stat, pval = mannwhitneyu(s1_vals, s2_vals, alternative = 'greater')
        pvals.append(pval)
    return pvals

In [18]:
pvals_20 = np.array(run_sims(num_annotators = 20, prob_diff = prob_diff))

 48%|████▊     | 4793/10000 [17:38<19:12,  4.52it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [19]:
pvals_25 = np.array(run_sims(num_annotators = 25, prob_diff = prob_diff))

 30%|██▉       | 2999/10000 [13:18<30:59,  3.77it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [20]:
pvals_30 = np.array(run_sims(num_annotators = 30, prob_diff = prob_diff))

 30%|██▉       | 2999/10000 [15:30<36:06,  3.23it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [21]:
print('For Number of Annotators = 20')
alpha = 0.05
print(f"alpha {alpha:5}, power: {np.mean(pvals_20 < alpha):0.3f}")

For Number of Annotators = 20
alpha  0.05, power: 0.855


In [22]:
print('For Number of Annotators = 25')
alpha = 0.05
print(f"alpha {alpha:5}, power: {np.mean(pvals_25 < alpha):0.3f}")

For Number of Annotators = 25
alpha  0.05, power: 0.910


In [23]:
print('For Number of Annotators = 30')
alpha = 0.05
print(f"alpha {alpha:5}, power: {np.mean(pvals_30 < alpha):0.3f}")

For Number of Annotators = 30
alpha  0.05, power: 0.938
